## Introdução

A Solar é uma rede de farmácias que opera com mais de 3.000 estabelecimentos em 7 países. Atualmente, os gerentes de cada farmácia da rede Solar têm a tarefa de **prever suas vendas diárias com até seis semanas de antecedência**. As vendas nas lojas são influenciadas por vários fatores, incluindo promoções, competidores, feriados escolares e estaduais, sazonalidade e localidade. Com milhares de gerentes prevendo vendas com base em suas circunstâncias únicas, a precisão dos resultados pode ser bastante variada.

Para as previsões você deve considerar um ID, que é um número representando a combinação entre o ID da loja e uma data. Além do ID, sua submissão deve conter a previsão do volume de vendas. Leia atentamente a descrição dos principais campos nos arquivos fornecidos nesta competição.

#### Sobre os datasets

Os datasets contém dados históricos de vendas para 1.115 lojas da Solar. 

#### Objetivo
A tarefa é prever a coluna "Vendas" no conjunto de teste. Observe que algumas lojas no conjunto de dados foram temporariamente fechadas para reformas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
df_sales = pd.read_csv('dataset_treino.csv')
df_stores = pd.read_csv('lojas.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
df_sales.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [21]:
df_stores.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


### Descrição dos Campos

**ID** - um ID que representa uma tupla (Store, Date) dentro do conjunto de dados

**Store** - um ID único para cada loja

**Sales** - o volume de negócios de um determinado dia (é isso que você está prevendo)

**Customers** - o número de clientes em um determinado dia

**Open** - um indicador para saber se a loja estava aberta: 
- 0 = fechada
- 1 = aberta

**StateHoliday** - indica um feriado estadual. Normalmente, todas as lojas, com poucas exceções, estão fechadas nos feriados estaduais. Note que todas as escolas estão fechadas nos feriados e fins de semana. 
- a = feriado público
- b = feriado de Páscoa
- c = Natal
- 0 = Nenhum

**SchoolHoliday** - indica se a (Store, Date) foi afetada pelo fechamento de escolas públicas

**StoreType** - diferencia entre 4 modelos de lojas diferentes: a, b, c, d

**Assortment** - descreve um nível: 
- a = básico
- b = extra
- c = estendido

**CompetitionDistance** - distância em metros até a loja concorrente mais próxima

**CompetitionOpenSince[Month/Year]** - indica o ano e mês aproximado da hora em que o concorrente mais próximo foi aberto

**Promo** - indica se uma loja está executando uma promoção nesse dia

**Promo2** - promoção contínua e consecutiva para algumas lojas: 
- 0 = loja não está participando
- 1 = loja está participando

**Promo2Since[Year/Week]** - descreve o ano e a semana do calendário em que a loja começou a participar do Promo2

**PromoInterval** - descreve os intervalos consecutivos que o Promo2 é iniciado, indicando os meses em que a promoção é iniciada novamente. Por exemplo. "Feb, May, Aug, Nov" significa que cada rodada começa em fevereiro, maio, agosto, novembro de qualquer ano para aquela loja.

### Algumas perguntas a serem respondidas

- Volume de vendas ao longo dos anos
- Volume de vendas / número de clientes (saber se teve dias em que poucos clientes compraram mas gastaram mais)
- Identificar quais lojas e quando tiveram reformas

In [22]:
print('Total de vendas:', len(df_sales))
print('Total de lojas com vendas:', df_sales.Store.nunique())
print('Total de lojas:', len(df_stores))

Total de vendas: 1017209
Total de lojas com vendas: 1115
Total de lojas: 1115


Unificando os datasets pelo código da loja

In [24]:
df = pd.merge(df_sales, df_stores, on='Store')

In [25]:
len(df)

1017209